In [2]:
using FFTW: fft, ifft, ifftshift

In [3]:
function step!(c::Array{Float64, 2}, 
               ξ²::Array{Float64, 2}, 
               D::Float64, 
               δt::Float64)

    dims = [1, 2]

    ĉ = fft(c, dims)
    ĉ = exp.(-D .* ξ² .* δt) .* ĉ
    c = abs.(ifft(ĉ, dims))

    return c

end

function step!(c::Array{Float64, 2}, 
               ξ²::Array{Float64, 2}, 
               D::Float64, 
               δt::Float64,
               P)

    dims = [1, 2]

    ĉ = P * c
    ĉ = exp.(-D .* ξ² .* δt) .* ĉ
    c = abs.(P \ ̂c)

    return c

end

step! (generic function with 1 method)

In [7]:
abstract type AbstractProblem end

abstract type AbstractSolver end




struct FRAPProblem{T, S} <: AbstractProblem:
   
    c_m::Array{T, 2}
    c_im::Array{T, 2}
    D::T
    δt::T
    ξ²::Array{T,2}
    n_pixels::S
    
    function FRAPProblem(c₀::T, φₘ::T, D::T, δt::T, n_pixels::S) where {T, S}
    
        mobile   = φₘ c₀
        immobile = (1-φₘ) c₀
        fourier_grid = create_fourier_grid(n_pixels)
        
        
        return new{T}(mobile, immobile, D, δt, fourier_grid, n_pixels)
            
    end
    
end



function solve(problem::AbstractProblem, solver::AbstractSolver) end


function evolve(p::FRAPProblem, T::Int64, masks)
    
    @unpack c_m, c_im, D, δt, ξ², n_pixels = p
    
    # Create initial arrays with grid size
    c_m  = c_m .* ones((n_pixel, n_pixel))
    c_im = c_im .* ones((n_pixel, n_pixel))
    
    # Pre-allocate solution
    c = Array{Float64, 2}(undef, (n_pixel, n_pixel, T))
    
    # For each frame  
    for t ∈ 1:T
        
        # Make make one step of length δt
        c_m = c_m |> step!(ξ², D, δt)
       
        # Apply imaging bleach masks
        c_m  = c_m |> bleach!(masks)
        c_im = c_im |> bleach!(masks)
        
        # Save time evolution
        @inbounds c[:, :, t] = c_m + c_im
        
    end
    
    return c
    
    
end



LoadError: syntax: line break in ":" expression

In [8]:
function create_fourier_grid(n_pixels::Int64)
    # TODO: Check dimensions of grid
    # It isn't centralized, but perhaps that is not a problem?

    # Julia has no ndgrid or meshgrid function, out of principle
    # it would seem. Not "Julian" enough.
    x = range(-n_pixels ÷ 2, length=n_pixels)
    y = range(-n_pixels ÷ 2, length=n_pixels)

    # List comprehensions are very quick
    X = [j for i in x, j in y]
    Y = [i for i in x, j in y]

    X *= (2π / n_pixels)
    Y *= (2π / n_pixels)

    # Centre the transform
    ξ² = ifftshift(X.^2 + Y.^2)

    return ξ²
end

create_fourier_grid (generic function with 1 method)